In [1]:
import pandas as pd

In [2]:
path = '/opt/ml/input/data'

train = pd.read_csv(path + '/train_data.csv')
test = pd.read_csv(path + '/test_data.csv')

In [3]:
# train, test 병합. 모든 데이터 활용하기.
dat = pd.concat([train, test], axis = 0)
dat = dat.sort_values(by = ['userID', 'Timestamp'])

In [4]:
dat

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag
0,0,A060001001,A060000001,1,2020-03-24 00:17:11,7224
1,0,A060001002,A060000001,1,2020-03-24 00:17:14,7225
2,0,A060001003,A060000001,1,2020-03-24 00:17:22,7225
3,0,A060001004,A060000001,1,2020-03-24 00:17:29,7225
4,0,A060001005,A060000001,1,2020-03-24 00:17:36,7225
...,...,...,...,...,...,...
2266581,7441,A030071005,A030000071,0,2020-06-05 06:50:21,438
2266582,7441,A040165001,A040000165,1,2020-08-21 01:06:39,8836
2266583,7441,A040165002,A040000165,1,2020-08-21 01:06:50,8836
2266584,7441,A040165003,A040000165,1,2020-08-21 01:07:36,8836


In [5]:
data = dat[['userID', 'testId', 'answerCode']].copy().reset_index(drop=True)
data.columns = ['userID', 'itemID', 'rating']

In [6]:
# rating이 -1로 되어있는 문제를 맞추기 위해 사용.
_train = data[data['rating'] >= 0]
_test = data[data['rating'] < 0]

In [7]:
_train = _train.drop_duplicates(['userID', 'itemID'], keep='last')

In [8]:
_train

,userID,itemID,rating
5,0,A060000001,1
12,0,A060000003,1
19,0,A060000005,1
26,0,A060000007,1
32,0,A080000002,1
...,...,...,...
2526680,7440,A050000096,0
2526685,7440,A030000136,0
2526690,7440,A030000197,0
2526695,7441,A030000071,0


In [9]:
from surprise import Dataset
from surprise import Reader

reader = Reader(rating_scale=(0,1))
_train = Dataset.load_from_df(_train, reader)

In [10]:
_train

In [11]:
# trainset.build_full_trainset()

In [12]:
_trainset = _train.build_full_trainset()

In [13]:
_trainset

In [14]:
testset = [_test.iloc[i].to_list() for i in range(len(_test))]

In [15]:
from surprise import SVDpp

In [16]:
model = SVDpp(random_state=42)

In [17]:
model.fit(_trainset)

In [18]:
model.test(testset)

[Prediction(uid=3, iid='A050000133', r_ui=-1, est=0.47154459325418135, details={'was_impossible': False}),
 Prediction(uid=4, iid='A070000146', r_ui=-1, est=0.7021296484360057, details={'was_impossible': False}),
 Prediction(uid=13, iid='A070000111', r_ui=-1, est=0.4830518202337246, details={'was_impossible': False}),
 Prediction(uid=17, iid='A090000064', r_ui=-1, est=0.44598334549034657, details={'was_impossible': False}),
 Prediction(uid=26, iid='A060000135', r_ui=-1, est=0.30825128281662073, details={'was_impossible': False}),
 Prediction(uid=29, iid='A020000190', r_ui=-1, est=0.8611655853161702, details={'was_impossible': False}),
 Prediction(uid=45, iid='A040000136', r_ui=-1, est=0.676605737219065, details={'was_impossible': False}),
 Prediction(uid=53, iid='A040000140', r_ui=-1, est=0.2779911648683757, details={'was_impossible': False}),
 Prediction(uid=58, iid='A070000159', r_ui=-1, est=0.13143840097637136, details={'was_impossible': False}),
 Prediction(uid=64, iid='A070000146'

In [19]:
pred = model.test(testset)
print('prediction type: ', type(pred))
print('size: ', len(pred))

prediction type:  <class 'list'>
size:  744


In [20]:
output_list = []
for i in range(len(pred)):
    output_list.append(pred[i].est)

In [21]:
submission = pd.DataFrame(columns=['id','prediction'])

In [22]:
submission['id'] = list(i for i in range(744))
submission['prediction'] = output_list

In [23]:
submission.to_csv('./output/surprise_svd_column_changed.csv', index = False)

In [1]:
lst = [0,1,2,3,4]

In [3]:
import pandas as pd
lst = pd.DataFrame(lst)